# GMFs and LUTs

This notebook will show how to use [xsarsea.windspeed.models.Model](../basic_api.rst#xsarsea.windspeed.models.Model)

Models are functions (GMF) or lookup table (LUT) that returns a simulated sigma0 from incidence angle, wind speed, and wind direction relative to the antenna.
 
Models are used for wind invertion with [xsarsea.windspeed.invert_from_model](../basic_api.rst#xsarsea.windspeed.invert_from_model), but they also be used independently.
 


In [ ]:
import xsar
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# optional debug messages
import logging
logging.basicConfig()
logging.getLogger('xsarsea.windspeed').setLevel(logging.DEBUG) # or .setLevel(logging.INFO)

## Available models

Available models can be retrieved with [xsarsea.windspeed.available_models](../basic_api.rst#xsarsea.windspeed.available_models)

In [ ]:
windspeed.available_models()

## Available models

Adding gmfs_impl (analytical models)  with [xsarsea.windspeed.gmfs.GmfModel.activate_gmfs_impl](../basic_api.rst#xsarsea.windspeed.gmfs.GmfModel.activate)

In [ ]:
windspeed.gmfs.GmfModel.activate_gmfs_impl()

### Adding netcdf models (LUT)

Netcdf models are not available by default, because they needs to be loaded from external file with [xsarsea.windspeed.register_nc_luts](../basic_api.rst#xsarsea.windspeed.register_nc_luts)


In [ ]:
nc_luts_path = xsar.get_test_file('nc_luts_reduce')
windspeed.register_nc_luts(nc_luts_path)
windspeed.available_models()

## Addind CMOD7


In [ ]:
try : 
    path_cmod7 = xsar.get_test_file("cmod7_and_python_script")
    windspeed.register_cmod7(path_cmod7)
except Exception as e:
    print(e)

### Adding Sarwing models (LUT)

Sarwing model are not available by default, because they needs to be loaded from external file with [xsarsea.windspeed.register_sarwing_luts](../basic_api.rst#xsarsea.windspeed.register_sarwing_luts)

A basic subset of sarwing lut can be retrieved with `xsar.get_test_file('sarwing_luts_subset')`. 

To get full sarwing luts, download them from https://cyclobs.ifremer.fr/static/sarwing_datarmor/xsardata/sarwing_luts, or use path `/home/datawork-cersat-public/cache/project/sarwing/GMFS/v1.6` (ifremer only)


In [ ]:
sarwing_luts_subset_path = xsar.get_test_file('sarwing_luts_subset')
windspeed.register_sarwing_luts(sarwing_luts_subset_path)
windspeed.available_models()

## Using models

[Model](../basic_api.rst#xsarsea.windspeed.models.Model) instance can be retrieved with [xsarsea.windspeed.get_model](../basic_api.rst#xsarsea.windspeed.get_model) 

In [ ]:
cmod5 = windspeed.get_model('cmod5')
cmod5

Models can be used as a regular function because they have a `__call__` method.

Argument to `__call__` are `(incidence, wspd, phi)` (phi is optionnal for crosspol lut).

If arguments are 1d arrays, output shape will be `(incidence,.size wspd.size, phi.size)`

2d arrays input are only implemented for [GmfModel](../basic_api.rst#xsarsea.windspeed.gmfs.GmfModel). The output will have the same shape as the input.

In [ ]:
incidence = np.array([25,35,45])
wspd = np.array([5, 40])
phi = np.array([0, 45, 90])
cmod5(incidence, wspd, phi)

The full lut can also be retrieved with [Model.to_lut](../basic_api.rst#xsarsea.windspeed.models.Model.to_lut)


In [ ]:
cmod5.to_lut()

Man can play with **kwargs of [GmfModel(Model)](../basic_api.rst#xsarsea.windspeed.gmfs.GmfModel).

In [ ]:
# this will directly generate a LUT at high resolution
cmod5.to_lut(**{'wspd_step' : 0.1, 'phi_step' : 1.5, 'inc_step' : 0.5, 'resolution' : 'high'})


In [ ]:
# this will generate a lut at low resolution and then interpolate at high spedified resolution ('resolution'=None)
cmod5.to_lut(**{'wspd_step' : 0.1, 'phi_step' : 1.5, 'inc_step' : 0.5, 'resolution' : None})

It won't have the same impact on SarwingLutModel or [NcLutModel](../basic_api.rst#xsarsea.windspeed.models.NcLutModel) or [cmod7Model](../basic_api.rst#xsarsea.windspeed.gmfs.cmod7Model).

Indeed, these are saved at at desired format with a certain resolution. 

By specifying **kwargs, it forced to interpolate the gmf at the desired resolution.

In [ ]:
# here we specify the exact same params than the saved LUT have
windspeed.get_model('nc_lut_sarwing_lut_cmodms1ahw').to_lut(**{'wspd_step' : 0.1, 'phi_step' : 1, 'inc_step' : 0.1, 'resolution' : 'high'})

In [ ]:
# here we specify different params than the saved LUT have : interpolation is made
windspeed.get_model('nc_lut_sarwing_lut_cmodms1ahw').to_lut(**{'wspd_step' : 0.1, 'phi_step' : 1, 'inc_step' : 0.2, 'resolution' : 'high'})

## In practice
[xsarsea.windspeed.invert_from_model](../basic_api.rst#xsarsea.windspeed.invert_from_model) can be called with **kwargs.

We can use kwargs to force the use of high resolution Luts. 

If possible, use analytical_luts (**gmf_...**).

Else, better LUTs that has not be interpolated. 

We use kwargs = **{"wspd_step": 0.1, "inc_step": 0.1, "phi_step": 0.1, "resolution": "high"}**

## Adding models (writing your own GMF)


A *Geophysical Modeling Function* (GMF) is a function that return a simulated *sigma0* from wind condition and instrument incidence angle.

A new gmf function is registered with the decorator [@xsarsea.windspeed.gmfs.GmfModel.register](../basic_api.rst#xsarsea.windspeed.gmfs.GmfModel.register)

To register a new GMF with `xsarsea.windspeed`, you have to follow the following rules:

  * parameters are `(incidence, windspeed, phi)`
    * `incidence` is in degrees
    * `windspeed` is in m/s
    * `phi` is wind direction, in degrees, relative to antenna look (0 is downwind in the antenna direction)
    
       note that `phi` is mandatory. If the gmf doesn't need `phi`, you have to explicitely set `phi=None` kwarg. 
    
  * all parameters must be **float**. numpy array are not allowed. `xsarsea.windspeed` will vectorize the function with numba to allow numpy arrays.
  * allowed units are `linear` or `dB`
  * function name must start with `gmf_`

In [ ]:
@windspeed.gmfs.GmfModel.register(pol='VH', units='linear', defer=False)
def gmf_dummy(inc, wspd, phi=None): 
    a0 = 0.00013106836021008122
    a1 = -4.530598283705591e-06
    a2 = 4.429277425062766e-08
    b0 = 1.3925444179360706
    b1 = 0.004157838450541205
    b2 = 3.4735809771069953e-05
    

    a = a0 + a1 * inc + a2 * inc ** 2
    b = b0 + b1 * inc + b2 * inc ** 2
    
    sig = a * wspd ** b

    return sig

In [ ]:
gmf_dummy

In [ ]:
windspeed.get_model('gmf_dummy').to_lut()

## HH Luts

We also created HH LUTS using CMODs and Polarization Ratio (PR) Models : 

- PR "mouche1" from Mouche, A., Hauser, D., Kudryavtsev, V., and Daloze, J.-F. (2005). Multi-polarisation ocean radar
cross-section from envisat asar observations, airborne polarimetric radar measurements and empirical
or semiempirical models
& 

- PR "zhang"  from Zhang, B., Perrie, W., and He, Y. (2011). Wind speed retrieval from radarsat-2 quad-polarization images
using a new polarization ratio model. Journal of Geophysical Research: Oceans.

We simply used this equation and created the NcLutModels

$$
nrcs_{HH} = \frac{nrcs_{VV}}{PR}
$$

For CMOD5n, we created high resolution LUTS (0.1m/s, 0.1°, 1°).

For CMOD7, we created low resolution LUTS (base cmod7 resolution from files) and high resolution (from file + interpolation). 

Then we can directly use these luts 

In [ ]:
# loading low resolution & interpolating 
windspeed.get_model('nc_lut_gmf_cmod7_Rlow_hh_mouche1').to_lut(**{'wspd_step' : 0.1, 'phi_step' : 1, 'inc_step' : 0.1, 'resolution' : 'high'})
# or 
# loading high resolution
windspeed.get_model('nc_lut_gmf_cmod7_Rhigh_hh_mouche1').to_lut(**{'wspd_step' : 0.1, 'phi_step' : 1, 'inc_step' : 0.1, 'resolution' : 'high'})

## Model comparison

This example function can be used to compare models.

In [ ]:

def model_compare(compare_models):
    
    luts = [ windspeed.get_model(name).to_lut(units='dB')  for name in compare_models]
    
    if 'phi' not in luts[0].dims:
        kdims=['incidence']
        dim_range=dict(incidence=(17,50))
    else:
        kdims=['incidence', 'phi']
        dim_range=dict(incidence=(17,50), phi=(0,360))
    
    def model_curve(incidence, phi=None):
        if 'phi' not in luts[0].dims:
            sel = dict(incidence=incidence)
        else:
            sel = dict(phi=phi, incidence=incidence)
        return hv.Overlay(
            [
                hv.Curve(lut.sel(**sel, method='nearest'),'wspd','sigma0', label=lut.attrs['model']) for lut in luts
            ]
        )
    
    
    dmap = hv.DynamicMap(model_curve, kdims=kdims).opts(height=600, width=600)
    return dmap.redim.range(**dim_range)
    


In [ ]:
windspeed.available_models(pol='VH')

In [ ]:
model_compare([ 'gmf_dummy', 'nc_lut_sarwing_lut_cmodms1ahw'] )

In [ ]:
windspeed.available_models(pol='VV')

In [ ]:
model_compare([ 'gmf_cmod7', 'gmf_cmod5n', 'gmf_cmod5'])